# works with the forCLIPandas environment
However, still the problem is that when given only one caption it returns the probability as 1 when it should have given the similarity score

In [2]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

image = preprocess(Image.open("/users/pgrad/mittalan/Desktop/dissertation/dissertation2022/input.jpg")).unsqueeze(0).to(device)
text = clip.tokenize(["a cat"]).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    
    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("Label probs:", probs)  # prints: [[0.9927937  0.00421068 0.00299572]]

Label probs: [[1.]]


In [2]:
!pwd

/bin/bash: /users/pgrad/mittalan/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/users/pgrad/mittalan/Desktop/dissertation/dissertation2022


In [1]:
import os
import clip
import torch
from torchvision.datasets import CIFAR100

# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

# Download the dataset
cifar100 = CIFAR100(root=os.path.expanduser("~/.cache"), download=True, train=False)

# Prepare the inputs
image, class_id = cifar100[3637]
image_input = preprocess(image).unsqueeze(0).to(device)
text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in cifar100.classes]).to(device)

# Calculate features
with torch.no_grad():
    image_features = model.encode_image(image_input)
    text_features = model.encode_text(text_inputs)

# Pick the top 5 most similar labels for the image
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
values, indices = similarity[0].topk(5)

# Print the result
print("\nTop predictions:\n")
for value, index in zip(values, indices):
    print(f"{cifar100.classes[index]:>16s}: {100 * value.item():.2f}%")

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting /users/pgrad/mittalan/.cache/cifar-100-python.tar.gz to /users/pgrad/mittalan/.cache

Top predictions:

           snake: 65.19%
          turtle: 12.44%
    sweet_pepper: 3.85%
          lizard: 1.88%
       crocodile: 1.74%


In [6]:
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)
image = preprocess(Image.open("input.jpg")).unsqueeze(0).to(device)
text_inputs=torch.cat([clip.tokenize(c) for c in ['cat', 'dog', 'pigeon']]).to(device)
with torch.no_grad():
    image_features = model.encode_image(image_input)
    text_features = model.encode_text(text_inputs)

# Pick the top 5 most similar labels for the image
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)

In [7]:
similarity

tensor([[0.3271, 0.5229, 0.1498]], device='cuda:0', dtype=torch.float16)

In [3]:
import pandas

In [4]:
import pandas as pd

In [5]:
CLIPrank = pd.read_csv('/users/pgrad/mittalan/Desktop/dissertation/dissertation2022/forCLIPranking.csv')
CLIPrank

,Unnamed: 0,category,headline,authors,link,short descr,date,labelexplain,VizEntailment,image_caption(s),image_link(s),OFAcaption
0,0,POLITICS,Ireland Votes To Repeal Abortion Amendment In ...,Laura Bassett,https://www.huffingtonpost.com/entry/results-f...,Irish women will no longer have to travel to t...,2018-05-26,neutral because the caption talks about women ...,N,The landmark referendum heralds a new era for ...,https://img.huffingtonpost.com/asset/5b0865f81...,two people walking in front of a mural on a wall
1,1,POLITICS,"Jack Johnson Was Pardoned, But Taboo Sex Is St...",Jenavieve Hatch,https://www.huffingtonpost.com/entry/jack-john...,A new law to fight sex trafficking targets som...,2018-05-25,the caption describes the image itself,E,"Jack Johnson in 1920 with his second wife, Luc...",https://img.huffingtonpost.com/asset/5b08732d1...,a black and white photo of a man and a woman
2,2,CRIME,Rachel Dolezal Faces Felony Charges For Welfar...,Carla Herreria,https://www.huffingtonpost.com/entry/rachel-do...,"State prosecutors say almost $84,000 had been ...",2018-05-25,the caption talks about Rachel and there is a ...,N,"Rachel Dolezal reportedly received over $80,00...",https://img.huffingtonpost.com/asset/5b0774621...,a woman standing in front of a video camera
3,3,POLITICS,Bishop Michael Curry Joins Christian March To ...,Carol Kuruvilla,https://www.huffingtonpost.com/entry/bishop-mi...,"For Curry and others, being Christian means re...",2018-05-25,NaN,E,Bishop Michael Curry (C) leaves the National C...,https://img.huffingtonpost.com/asset/5b0830721...,a group of people walking down the street at a...
4,4,CRIME,"Man Faces Charges After Pulling Knife, Stun Gu...",Jenna Amatulli,https://www.huffingtonpost.com/entry/man-knife...,"“We thought we were going to die,"" one of the ...",2018-05-25,NaN,N,"John Jay Smith, 60, faces multiple felony char...",https://img.huffingtonpost.com/asset/5b0857ea1...,a screenshot of a news report with a man sayin...
...,...,...,...,...,...,...,...,...,...,...,...,...
119,119,SPORTS,NBA Star Reveals Struggle With Panic Attacks A...,Carla Herreria,https://www.huffingtonpost.com/entry/kevin-lov...,Kevin Love of the Cavaliers wants men to know ...,2018-03-07,NaN,N,Kevin Love of the Cleveland Cavaliers wrote of...,https://img.huffingtonpost.com/asset/5a9f18ba1...,a basketball player is standing in a crowd
120,120,SPORTS,People Sent A Lot Of Racist Email To UCLA Afte...,Travis Waldron,https://www.huffingtonpost.com/entry/ucla-raci...,“3 JERKS — DISGRACED THE COLLEGE — OUR COUNTRY...,2018-03-06,NaN,E,"Cody Riley (left), LiAngelo Ball and Jalen Hil...",https://img.huffingtonpost.com/asset/5a9d83591...,three young men sitting at a table with a micr...
121,121,SPORTS,"Roger Bannister, First Man To Run Mile In Unde...",NaN,https://www.huffingtonpost.com/entry/roger-ban...,Roger Bannister made headlines around the worl...,2018-03-04,NaN,N,ir Roger was a 25-year-old Oxford University m...,https://img.huffingtonpost.com/asset/5a9c04af1...,a man in a suit walking on a race track
122,122,SPORTS,Aly Raisman Sues U.S. Olympic Committee For Si...,Willa Frej,https://www.huffingtonpost.com/entry/aly-raism...,"""I refuse to wait any longer for these organiz...",2018-03-02,NaN,E,Olympic gold medalist Aly Raisman has filed a ...,https://img.huffingtonpost.com/asset/5a9947f91...,a woman standing in front of a microphone


In [8]:
import torch
import clip
import urllib.request
from PIL import Image
prob=[]
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)
for i, row in CLIPrank[1:10].iterrows():
    urllib.request.urlretrieve(row['image_link(s)'],"gfg.png")
    image = Image.open("gfg.png")
#     image = preprocess(Image.open("/users/pgrad/mittalan/Desktop/dissertation/dissertation2022/input.jpg")).unsqueeze(0).to(device)
    text = clip.tokenize([row['headline'], row['image_caption(s)'], row['OFAcaption'], row['short descr']]).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)

        logits_per_image, logits_per_text = model(image, text)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()
    prob.append(probs)

AttributeError: type